# L'approche fonctionnelle

---

- De l'avis des puristes du monde d'Haskell ou du Lisp, Rust n'est pas à proprement parler un langage fonctionnel, mais pourtant, il s'inspire largement de ces deux langages précédents dans nombre d'aspects. Nous ne débattrons pas ici de savoir si Rust est vraiment un langage fonctionnel ou pas, car tout dépend de la définition que nous donnerions à un langage fonctionnel. Voici un extrait de la page Wikipédia de la programmation fonctionnel :
> La programmation fonctionnelle est un paradigme de programmation de type déclaratif qui considère le calcul en tant qu'évaluation de fonctions mathématiques.  
> Comme le changement d'état et la mutation des données ne peuvent pas être représentés par des évaluations de fonctions, la programmation fonctionnelle ne les admet pas, au contraire elle met en avant l'application des fonctions, contrairement au modèle de programmation impérative qui met en avant les changements d'état.

*Sachez que pour les puristes de tout à l'heure, Rust n'est pas un langage à proprement parler fonctionnel, puisqu'il n'est pas encore tout à fait optimisé pour la récursivité terminale...*

Nous avons déjà croisé plusieurs aspects de la programmation fonctionnelle dans ce cours :

- If let et le pattern matching
- L'opérateur Try, les Option et les Result
- Les iterateurs
- Les closures

**Ici, nous allons revenir plus en profondeur sur la notion d'iterateur et ce qu'ils permettent.**

## Présentation des Iterateurs

- Les itérateurs permettent d'effectuer une tâche sur une séquence d'éléments à tour de rôle. En Rust, un itérateur est une **évaluation paresseuse** (lazy evaluation en anglais), ce qui signifie qu'il n'a aucun effet jusqu'à ce que nous appelions des méthodes qui consomment l'itérateur pour l'utiliser.

On peut considérer un Iterateur comme un `distributeur d'éléments` :

In [3]:
{
    let prime_number_list = [2, 3, 5, 7, 11, 13];     // Tableau de 6 nombres premiers

    let my_prime_iterator = prime_number_list.iter(); // Creation d'un iterateur sur le tableau via la methode iter()
    for prime_number in my_prime_iterator {           // On parcours l'iterateur
        dbg!(prime_number);                           // Affiche l'element courant
    }
}

[src/lib.rs:125] prime_number = 2
[src/lib.rs:125] prime_number = 3
[src/lib.rs:125] prime_number = 5
[src/lib.rs:125] prime_number = 7
[src/lib.rs:125] prime_number = 11
[src/lib.rs:125] prime_number = 13


()

- Il n'est pas obligatoire de créer explicitement l'iterateur via la méthode iter() dans le cas d'un parcours avec la boucle for pour une slice :

In [2]:
let fizzbuzz = ["Fizz", "Buzz", "FizzBuzz"];
for word in fizzbuzz {
    println!("{:?}", word);
}

"Fizz"
"Buzz"
"FizzBuzz"


()

## Principe interne de l'iterateur

- Bien que ce ne soit pas visible avec la boucle for, a chaque itération, la **methode next()** du trait Iterator est appelée. Cette méthode retourne une Option qui vaudra soit Some(mon_type) pour un élément ou bien None quand il n'y a plus d'élément dans l'iterateur. Cet exemple l'illustre bien :

In [4]:
{
    let fizzbuzz = ["Fizz", "Buzz", "FizzBuzz"];
    let mut iter = fizzbuzz.iter(); // Creation d'un iterateur sur fizzbuzz. La structure Iter implement Iterator
    println!("{:?}", iter.next()); // Premier element
    println!("{:?}", iter.next()); // Second element
    println!("{:?}", iter.next()); // Troisieme element
    println!("{:?}", iter.next()); // L'iterateur est vide
}

Some("Fizz")
Some("Buzz")
Some("FizzBuzz")
None


()

- On peut ainsi obtenir le même comportement qu'avec la boucle for avec while let :

In [5]:
{
    let fizzbuzz = ["Fizz", "Buzz", "FizzBuzz"];
    let mut iter = fizzbuzz.iter(); // Creation d'un iterateur sur fizzbuzz
    while let Some(word) = iter.next() {
        println!("{}", word);
    }
}

Fizz
FizzBuzz
Buzz


()

> Nous remarquons que la boucle **for in** n'est qu'un simple sucre syntaxique qui fonctionne en réalité via une boucle while, les iterateurs et les Options.

- On trouve dans le document du Trait std::iter::Iterator cette méthode next() en **Required Methods** :

![ITER](pictures/iter_prototype.png)

## Exemple d'implémentation d'un Iterateur "custom"

> Si je veux créer un Iterateur à partir d'un de mes propres types telle une structure ou une énumération, je devrais :
> - Créer un nouveau type de structure qui contient une référence (mutable ou non) sur mon type, dans ce cas, on a besoin de spécifier les lifetime.
> - Implémenter le trait Iterator pour cette structure en définissant la méthode next()
> - Implémenter une méthode iter() pour ma structure de départ qui génère ma seconde structure qui est l'iterateur. 

In [6]:
struct MyStruct { // Ma structure de base
    i: usize,
    j: usize,
    k: usize,
}
struct MyIterator<'a> { // La structure de mon iterateur
    r: &'a MyStruct,    // Reference sur ma structure de base
    count: usize,       // Compteur pour savoir ou j'en suis durant l'iteration
}
impl Iterator for MyIterator<'_> {                 // Implementation du trait Iterator pour la structure de l'iterateur
    type Item = usize;                             // Mon type d'element
    fn next(&mut self) -> Option<Self::Item> {     // Implementation obligatoire de la methode next()
        let output = match self.count {
            0 => Some(self.r.i),
            1 => Some(self.r.j),
            2 => Some(self.r.k),
            _ => None                               // Quand il n'y a plus rien, on retourne None
        };
        self.count += 1;                            // Je ne dois pas oublier ca !
        output
    }
}
impl MyStruct {
    fn iter(&self) -> MyIterator { // La methode iter() fait retourne une structure qui me permet d'iterer
        MyIterator {
            r: self,               // Reference sur self
            count: 0,              // Le compteur doit bien etre init a 0 !
        }
    }
}
let my_struct = MyStruct {
    i: 2,
    j: 3,
    k: 4
};
for value in my_struct.iter() {
    dbg!(value);
}

[src/lib.rs:156] value = 2
[src/lib.rs:156] value = 3
[src/lib.rs:156] value = 4


()

*Cet exemple n'a pas vocation à être entièrement compris durant ce cours.*

## Les méthodes sur les Iterateurs

> La documentation Rust du Trait std::iter::Iterator nous indique qu'il existe une myriade de méthodes qui fonctionnent sur les iterateurs et c'est la tout leur intérêt. Il est possible d'appliquer des fonctions mathématiques sur les éléments, de modifier l'iterateur ou de créer de nouveaux iterateurs à partir d'iterateurs.

### Comparaison du code impératif et du code fonctionnel

- Prenons cet exemple de code impératif :

In [7]:
let v = vec![1, 2, 3];
let mut total = 0;
for i in 0..v.len() {
    total += v[i]; // On additionne tous les elements de la collection v
}
dbg!(total);

[src/lib.rs:143] total = 6


- Cet exemple peut se réécrire ainsi avec les iterateurs :

In [8]:
let v = vec![1, 2, 3];
let total = v.iter().fold(0, |total, next| total + next);
dbg!(total);

[src/lib.rs:140] total = 6


> On notera qu'il n'y a plus de variable mutable total. On dit qu' `il n'y a pas d'effet de bord` avec les iterateurs. L'incrémentation de la variable total du premier exemple est un effet de bord.
> Ainsi, nombre de programmeurs trouvent l'expression sous forme fonctionnelle bien plus élégante et sure.

### Un mot sur l'évaluation paresseuse

- Ici, l'iterateur ne sera "consomme" que lors de l'utilisation de la méthode collect()... Collect genere une nouvelle collection à partir d'un iterateur :

In [23]:
{
    println!("Start");
    let squared: std::iter::Map<_, _> = (1..=3).map(|x| { // Creation d'un nouvel Iterateur a partir de Range
        println!("One more loop...");
        x * x
    });
    println!("After iterator definition and before collect");
    let v: Vec<u32> = squared.collect();                   // Evaluation de l'iterateur ici
    dbg!(v);
    println!("End");
}

Start
After iterator definition and before collect
One more loop...
One more loop...
One more loop...
End


[src/lib.rs:164] v = [
    1,
    4,
]
    9,


()

## Quelques exemples d'utilisation des iterateurs

### Les filtres

* Il est possible de ne retenir que certaines valeurs sur les iterateurs :

In [25]:
{
    let v: Vec<u32> = vec![1, 2, 7, 11, 12];
    let my_new_iterator = v.iter().filter(|elem| *elem % 2 == 0); // Ne retient que les nombres pairs
    
    for elem in my_new_iterator {
        dbg!(elem);
    }
}

[src/lib.rs:161] elem = 2
[src/lib.rs:161] elem = 12


()

> La méthode filter du trait Iterator prend en paramètre une fonction retournant true ou false, le prédicat. La sortie de la méthode filter est une structure de type Filter qui implémente à son tour le trait Iterator :
```
fn filter<P>(self, predicate: P) -> Filter<Self, P> 
where
    Self: Sized,
    P: FnMut(&Self::Item) -> bool,
```

### Le mapping

- Map permet, entre autres, d'appliquer une fonction mathématique sur chaque élément de l'iterateur et/ou de transmuter le type des elements :

In [7]:
{
    let v: Vec<u32> = vec![1, 2, 3];
    let my_new_iterator = v.iter().map(|elem| *elem * 2);
    for elem in my_new_iterator {
        dbg!(elem);
    }
}

[src/lib.rs:156] elem = 2
[src/lib.rs:156] elem = 4
[src/lib.rs:156] elem = 6


()

> La méthode map du trait Iterator prend en paramètre une fonction retournant un nouvel élément qui peut être de type diffèrent de celui en entrée. La sortie de la méthode map est une structure de type Map qui implémente à son tour le trait Iterator :
```
fn map<B, F>(self, f: F) -> Map<Self, F>
where
    Self: Sized,
    F: FnMut(Self::Item) -> B,
```

In [9]:
{
    // Exemple de casting de type de tous les elements pour v2 depuis v1
    let v1: Vec<u32> = vec![1, 2, 3];
    let v2: Vec<u64> = v1.iter().map(|value| *value as u64).collect();
    // assert_eq!(v1, v2); // Ne peut comparer des u32 et u64
    dbg!(v2);
}

[src/lib.rs:157] v2 = [
    1,
    2,
    3,
]


()

> **NB** N'oubliez pas que le paramètre d'input de la closure ici value est une référence (parce que FnMut) et qu'il est fréquent de devoir la déréférencer via `*` pour l'utiliser.

### Fusionner deux iterateurs :

- zip permet de fusionner deux iterateurs. L'iterateur résultant contiendra des tuples :

In [32]:
{
    let iter_1 = (0..2);                        // Iterateur a partir d'une Range
    let iter_2 = (10..12);                      // Iterateur a partir d'une Range
    let my_new_iterator = iter_1.zip(iter_2);
    for elem in my_new_iterator {
        dbg!(elem);
    }
}

[src/lib.rs:161] elem = (
    0,
    10,
)
[src/lib.rs:161] elem = (
    1,
    11,
)


()

### Les Iterateurs mutables

- Enfin, il n'est pas toujours nécessaire de générer de nouveaux iterateurs. Les éléments à l'intérieur d'un iterateur peuvent être directement modifies. Ceci peut être réalisé par l'utilisation conjointe de iter_mut() et de la méthode for_each() :

In [34]:
{
    let mut v = vec![0, 1, 2];
    v.iter_mut().for_each(|mut elem| *elem *= 2);
    dbg!(v);
}

[src/lib.rs:159] v = [
    0,
    2,
    4,
]


()

> Contrairement à map ou a filter, for_each ne retourne pas de nouvel iterateur, il se contente d'exécuter une procédure pour chaque élément de l'iterateur et peut éventuellement modifier les données référencées par l'iterateur s'il est utilisé avec iter_mut() (iterateur mutable).
```
fn for_each<F>(self, f: F)
where
    Self: Sized,
    F: FnMut(Self::Item),
```

> **N'hésitez pas à consulter la documentation du trait Iterator afin d'entrevoir toutes les possibilités qu'offrent les itérateurs. Notez aussi que certaines des méthodes présentées ici telles `map` peuvent aussi s'utiliser pour les `Option<T>` et les `Result<T, E>`, tout est dans la documentation `rustup doc`.**